# styles
Some css items that work for both svg and html

In [ ]:
#| default_exp styles

### Setup

In [ ]:
#| export
import numpy as np

from dataclasses import dataclass
from collections import namedtuple
from typing import Optional, Literal

from fastcore.basics import patch
from collections import namedtuple

#For parsing patterns
import xml.etree.ElementTree as ET
from abc import ABC, abstractmethod
from bs4 import BeautifulSoup
import lxml

# fun with colors
import colorsys
import seaborn as sns
import matplotlib.pyplot as plt


from IPython.display import SVG, HTML

In [ ]:
from nbdev import show_doc

#| export
### Helpers
We want to be able to build svg and have put many of the common styling information into a demo

In [ ]:
#| export
class StyleDemo:
    def __init__(self):
        self.help = ""

## Small building blocks

SVG, tags. like HTML, is made up of tags with attributes. We need some simple classes that will let us print tags quickly


In [ ]:
#| export
def tag(name, content, **attributes):
    if attributes:
        attrs = ' '.join(f'{k.replace("_", "-")}="{v}"' for k, v in attributes.items())
        return f"<{name} {attrs}>{content}</{name}>"
    return f"<{name}>{content}</{name}>"

def indent(text, level=1):
    return '\n'.join('\t' * level + line for line in text.split('\n'))

class Generatable(ABC):
    """ SVG, tags. like HTML, is made up of tags with attributes. We need a simple abstract class that will let us print tags quickly. 
    The generate function is so that things can be dynamically computed.
    
    """
    @abstractmethod
    def generate(self) -> str:
        pass

    def tag(self,name) -> str:
        return tag(name,self.generate())


In [ ]:
#| export
NamedColor = namedtuple('NamedColor', ['color', 'name'])



In [ ]:
#| export
class SVGDef(Generatable):
    """ This is the concrete class that is the simplest implementation of Generatable. It just stores the content in a string. More complex version of Generatable will compute this on the fly ."""

    def __init__(self, tag, id, content,  **attributes): 
        self.content = content
        self.tag_name = tag  # Renamed to avoid shadowing the class method
        if attributes is None:
            attributes = {}
        attributes["id"] = id
        self.attributes = attributes

    def generate(self):
        return tag(self.tag_name, self.content, **self.attributes)


    def display(self):
        ret = f"+{self.tag_name} Header\n"

        for key in sorted(self.attributes.keys()):
            value = self.attributes[key]
            
            ret += f"\t{key}:{value}"
            ret += "\n"
        ret += f"-{self.tag_name} Header\n\n"
        ret += f"+{self.tag_name} Body\n"
        ret += self.content
        ret += f"\n-{self.tag_name} Body\n"

        return ret

    def __str__(self):
        return self.generate()

    def __repr__(self):
        return self.display()


In [ ]:
str(SVGDef("clipPath", "hex-clip", "Rect()", clipPathUnits="objectBoundingBox"))


In [ ]:
#| export
class StyleCSS(Generatable):
    """CSS style class for SVG elements with support for pseudo-classes and transformations.
    
    StyleCSS provides a programmatic way to create and manage CSS styles for SVG elements,
    including support for hover effects, color manipulation, and preset color palettes.
    
    Attributes:
        name (str): CSS class name (will be prefixed with '.' in generated CSS)
        properties (dict): Dictionary of CSS properties (underscores converted to hyphens)
        pseudo_styles (dict): Dictionary mapping pseudo-class names to their property dicts
    
    Examples:
        Basic style creation:
        >>> water = StyleCSS("water", fill="#0D47A1", stroke="#333", stroke_width=2)
        
        With hover effect:
        >>> hills = StyleCSS("hills", fill="#D4E157", stroke="#333")
        >>> hover = StyleCSS("hover_effect", fill="#007fff", cursor="pointer")
        >>> hills.customize(hover)
        
        Generate CSS for multiple styles:
        >>> styles = [water, hills]
        >>> css = StyleCSS.generate(styles)
        
        Color manipulation:
        >>> light_hills = hills.lighten(0.3)
        >>> faded = hills.with_opacity(0.5)
        >>> muted = hills.desaturate(0.6)
        
        Preset palettes:
        >>> elevation_styles = StyleCSS.elevations()  # 9 terrain elevation colors
        >>> rocket_styles = StyleCSS.seaborn("rocket", 7)  # Seaborn color palette
        
        Copy and modify:
        >>> new_style = hills.copy("hills_variant")
        >>> new_style.properties["fill"] = "#FF0000"
    
    Class Methods:
        generate(styles): Generate CSS string from list of StyleCSS objects
        from_style_string(style_str, counter, existing_styles): Parse inline style to StyleCSS
        elevations(): Return preset terrain elevation color palette (9 levels)
        seaborn(name, levels): Generate styles from Seaborn color palette
        
    Instance Methods:
        customize(style_obj, pseudo_class): Add pseudo-class style (e.g., :hover)
        desaturate(factor): Reduce color saturation
        lighten(factor): Increase color luminance
        with_opacity(opacity): Create copy with modified opacity
        copy(new_name): Create a copy with optional new name
        
    Notes:
        - Property names use underscores which are automatically converted to hyphens
          (e.g., stroke_width becomes stroke-width)
        - The generate() method handles both main styles and pseudo-class styles
        - Color manipulation methods return new StyleCSS objects, leaving originals unchanged
    """

    def __init__(self, name, **properties):
        self.name = name
        self.properties = properties
        self.pseudo_styles = {}
    
    def customize(self, style_obj, pseudo_class="hover"):
        """Add a pseudo-class style (e.g., :hover, :active, :focus)"""
        self.pseudo_styles[pseudo_class] = style_obj.properties
        return self
    
    @staticmethod
    def _format_property(key):
        """Convert underscores to hyphens for CSS property names"""
        return key.replace('_', '-')
    
    @staticmethod
    def _format_properties(properties, indent=2):
        """Format properties as CSS with proper indentation"""
        lines = []
        for key, value in properties.items():
            prop_name = StyleCSS._format_property(key)
            lines.append(f"{' ' * indent}{prop_name}:{value};")
        return '\n'.join(lines)
    
    @classmethod
    def generate(cls, styles):
        """Generate CSS string from a list of StyleCSS objects"""
        css_blocks = []
        
        for style in styles:
            # Main style block
            block = f".{style.name} {{\n"
            block += cls._format_properties(style.properties)
            block += "\n}"
            css_blocks.append(block)
            
            # Pseudo-class blocks
            for pseudo, props in style.pseudo_styles.items():
                pseudo_block = f".{style.name}:{pseudo} {{\n"
                pseudo_block += cls._format_properties(props)
                pseudo_block += "\n}"
                css_blocks.append(pseudo_block)
        
        return '\n'.join(css_blocks)

    @classmethod
    def from_style_string(cls, style_str, counter, existing_styles=None):
        """Create StyleCSS from inline style string, checking for duplicates"""
        if existing_styles is None:
            existing_styles = []
        
        # Parse style string
        props = {}
        for item in style_str.split(';'):
            if ':' in item:
                key, value = item.split(':', 1)
                props[key.strip().replace('-', '_')] = value.strip()
        
        # Check for duplicate (compare fill and stroke_width only)
        fill = props.get('fill')
        width = props.get('stroke_width') or props.get('width')
        
        for existing in existing_styles:
            existing_fill = existing.properties.get('fill')
            existing_width = existing.properties.get('stroke_width') or existing.properties.get('width')
            if fill == existing_fill and width == existing_width:
                return existing, False  # Return existing style, not new
        
        # Create new style
        name = f"pat_fill_{counter}"
        return cls(name, **props), True

    @classmethod
    def elevations(cls):
        els = [("#66BB6A","Rich lowland green","Lowland"),
        ("#9CCC65","Bright plains green","Plains"),
        ("#D4E157","Yellow-green hills","Hills"),

        ("#FFCA28","Golden Highlands","Foothills"),
        ("#FF9800","Orange mountains","Highlands"),
        ("#F57C00","Dark orange peaks","Mountains"),
        ("#E65100","Red-orange summits","Summits"),
        ("#BF360C","Deep red peaks","Peaks"), 
        ("#ffffff","snow","Snow")]
        ret = []
        for elevation in els:
            ret.append(StyleCSS(elevation[2],stroke="#333333",stroke_width=1.5,opacity=0.9,fill=elevation[0]))
        return ret

    @classmethod
    def seaborn(cls,name:str,levels=7):
        palette = sns.color_palette(name, levels)
        hex_colors = [plt.matplotlib.colors.rgb2hex(c) for c in palette]
        ret = []
        for i, color in enumerate(hex_colors):
            ret.append(StyleCSS(f"{name}{i}" ,stroke="#333333",stroke_width=1.5,fill = color))
        return ret

In [ ]:
hills = StyleCSS("Yellow_green_hills", fill="#D4E157", stroke="#333333", stroke_width=1.5, stroke_opacity=0.9)
hSmall = StyleCSS("hoover",fill="#007fff",cursor= "pointer")
hills.customize(hSmall)
print("some styles")
print(tag("style",StyleCSS.generate([hills])))

In [ ]:
StyleCSS.generate(StyleCSS.elevations())

In [ ]:
someStyles = [StyleCSS.from_style_string("fill:#66BB6A;stroke:#333333",1)[0]]
StyleCSS.generate(someStyles)

In [ ]:
print(StyleCSS.generate(StyleCSS.seaborn("rocket",14)))

In [ ]:
#| export
@patch
def desaturate(self:StyleCSS,factor:float =0.5):
    if "fill" not in self.properties:
        return "#ffffff"

    hex_color = self.properties["fill"]

    if not hex_color.startswith("#"):
        return self

    hex_color = hex_color.lstrip('#')

    # Convert hex to RGB (0-1 range)
    r, g, b = tuple(int(hex_color[i:i+2], 16) / 255 for i in (0, 2, 4))
    
    # Convert to HSL
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    
    # Reduce saturation
    s *= factor
    
    # Convert back to RGB
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    
    # Convert to hex
    return f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}"

In [ ]:
#| export
@patch
def lighten(self: StyleCSS, factor: float = 0.2):
    """Lighten the fill color by increasing luminance"""
    if "fill" not in self.properties or not self.properties["fill"].startswith("#"):
        return self
    
    hex_color = self.properties["fill"].lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) / 255 for i in (0, 2, 4))
    h, l, s = colorsys.rgb_to_hls(r, g, b)
    l = min(1.0, l + factor)
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    
    new_color = f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}"
    new_style = StyleCSS(f"{self.name}_light", **self.properties)
    new_style.properties["fill"] = new_color
    return new_style

@patch
def with_opacity(self: StyleCSS, opacity: float):
    """Create a new style with modified opacity"""
    new_style = StyleCSS(f"{self.name}_o{int(opacity*10)}", **self.properties)
    new_style.properties["opacity"] = opacity
    return new_style

@patch
def copy(self: StyleCSS, new_name: str = None):
    """Create a copy of the style with optional new name"""
    name = new_name or f"{self.name}_copy"
    new_style = StyleCSS(name, **self.properties.copy())
    new_style.pseudo_styles = self.pseudo_styles.copy()
    return new_style

In [ ]:
def gradient_palette(cls, name: str, start_color: str, end_color: str, steps: int = 7):
    """Generate gradient between two colors"""
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    
    def rgb_to_hex(rgb):
        return f"#{int(rgb[0]):02x}{int(rgb[1]):02x}{int(rgb[2]):02x}"
    
    start_rgb = hex_to_rgb(start_color)
    end_rgb = hex_to_rgb(end_color)
    
    styles = []
    for i in range(steps):
        factor = i / (steps - 1) if steps > 1 else 0
        r = start_rgb[0] + (end_rgb[0] - start_rgb[0]) * factor
        g = start_rgb[1] + (end_rgb[1] - start_rgb[1]) * factor
        b = start_rgb[2] + (end_rgb[2] - start_rgb[2]) * factor
        
        color = rgb_to_hex((r, g, b))
        styles.append(cls(f"{name}_{i}", stroke="#333333", stroke_width=1.5, fill=color))
    
    return styles

@classmethod
def monochrome_palette(cls, name: str, base_color: str, steps: int = 7):
    """Generate monochrome palette from a base color"""
    return cls.gradient_palette(name, "#ffffff", base_color, steps)

## Amimation

In [ ]:
#| export
class LoopingLayerAnimation(Generatable):
    def __init__(self, 
                 layers: list[str],
                 visible_count: int = 2,
                 step_duration: float = 0.5,
                 fade_duration: float = 0.1,
                 dim_opacity: float = 0.0):
        self.layers = layers
        self.visible_count = visible_count
        self.step_duration = step_duration
        self.fade_duration = fade_duration
        self.dim_opacity = dim_opacity
    
    def _compute_timing(self):
        """Returns list of visible layer indices for each step"""
        counters = np.arange(self.visible_count)
        timing = []
        for i in range(len(self.layers)):
            timing.append(list(counters))
            counters = (counters + 1) % len(self.layers)
        return timing
    
    def _compute_events(self, timing):
        """Returns dict mapping layer_idx -> list of ('in'/'out', start_time, end_time)"""
        layer_events = {i: [] for i in range(len(self.layers))}
        prev_visible = set(timing[0])
        
        # Handle wrap-around: compare timing[-1] to timing[0]
        all_steps = list(range(1, len(timing))) + [0]
        
        for step_idx in all_steps:
            current_visible = set(timing[step_idx])
            step_time = step_idx * self.step_duration
            if step_idx == 0:
                step_time = len(timing) * self.step_duration  # wrap time
            fade_start = step_time - self.fade_duration
            
            # Arrivals
            for layer_idx in current_visible - prev_visible:
                layer_events[layer_idx].append(('in', fade_start, step_time))
            
            # Departures
            for layer_idx in prev_visible - current_visible:
                layer_events[layer_idx].append(('out', fade_start, step_time))
            
            prev_visible = current_visible
        
        return layer_events
    
    def _events_to_keyframes(self, layer_idx, events, timing):
        """Convert events list to CSS keyframe string"""
        total_duration = len(self.layers) * self.step_duration
        
        # Build list of (time_pct, opacity) points
        points = []
        
        for event_type, start_time, end_time in events:
            start_pct = (start_time / total_duration) * 100
            end_pct = (end_time / total_duration) * 100
            
            if event_type == 'in':
                points.append((start_pct, self.dim_opacity))
                points.append((end_pct, 1.0))
            else:  # out
                points.append((start_pct, 1.0))
                points.append((end_pct, self.dim_opacity))
        
        # Sort by time
        points.sort(key=lambda x: x[0])
        
        # Build keyframe CSS
        keyframe_lines = [f"  {pct:.2f}% {{ opacity: {op}; }}" for pct, op in points]
        
        layer_name = self.layers[layer_idx]
        keyframe_name = f"loop-{layer_name}"
        return f"@keyframes {keyframe_name} {{\n" + "\n".join(keyframe_lines) + "\n}"

        #return f"@keyframes {keyframe_name} {{\n" + "\n".join(keyframe_lines) + "\n}}"
    
    def _generate_animation_rule(self, layer_idx):
        """Generate the animation property CSS for a layer"""
        total_duration = len(self.layers) * self.step_duration
        layer_name = self.layers[layer_idx]
        keyframe_name = f"loop-{layer_name}"
        return f'g[data-layer="{layer_name}"] {{ animation: {keyframe_name} {total_duration}s linear infinite; }}'
    
    def generate(self) -> str:
        """Generate all @keyframes and animation rules"""
        timing = self._compute_timing()
        events = self._compute_events(timing)
        
        css_parts = []
        
        for layer_idx in range(len(self.layers)):
            if events[layer_idx]:  # only if there are events
                css_parts.append(self._events_to_keyframes(layer_idx, events[layer_idx], timing))
                css_parts.append(self._generate_animation_rule(layer_idx))
        
        return "\n".join(css_parts)
    
    def initial_opacities(self) -> dict[str, float]:
        """Returns dict of layer_name -> initial opacity for setup"""
        timing = self._compute_timing()
        initial_visible = set(timing[0])
        return {
            self.layers[i]: 1.0 if i in initial_visible else self.dim_opacity
            for i in range(len(self.layers))
        }


In [ ]:
#| export
def apply_looping_animation(builder, anim: LoopingLayerAnimation):
    """Apply a LoopingLayerAnimation to an SVGBuilder"""
    # Set initial opacities
    for layer_name, opacity in anim.initial_opacities().items():
        for layer in builder.layers:
            if layer.name == layer_name:
                layer.properties['opacity'] = str(opacity)
                break
    
    # Add animation CSS
    if not hasattr(builder, 'custom_animations'):
        builder.custom_animations = []
    builder.custom_animations.append(anim.generate())

In [ ]:
#| export
@dataclass
class LayerAnimation:
    """Configuration for a single layer's animation."""
    
    # Animation type
    animation_type: Literal["fade", "slide", "scale", "none"] = "fade"
    
    # Timing
    duration: float = 1.0  # seconds
    delay: float = 0.0  # seconds
    
    # Easing function
    easing: str = "ease-in-out"  # CSS easing function
    
    # Initial state (for fade/scale)
    start_opacity: float = 0.0
    end_opacity: float = 1.0
    
    # Slide animation (if animation_type="slide")
    slide_from: Optional[tuple[float, float]] = None  # (x, y) offset
    
    # Scale animation (if animation_type="scale")
    start_scale: float = 0.8
    end_scale: float = 1.0
    
    # Transform origin (for scale/rotate)
    transform_origin: str = "center center"
    
    # Animation options
    fill_mode: str = "forwards"  # Keep final state
    iteration_count: int = 1  # Run once

In [ ]:
#| export
EASING_PRESETS = {
    # Standard
    "linear": "linear",
    "ease": "ease",
    "ease-in": "ease-in",
    "ease-out": "ease-out",
    "ease-in-out": "ease-in-out",
    
    # Custom cubic-bezier curves
    "ease-in-quad": "cubic-bezier(0.55, 0.085, 0.68, 0.53)",
    "ease-out-quad": "cubic-bezier(0.25, 0.46, 0.45, 0.94)",
    "ease-in-out-quad": "cubic-bezier(0.455, 0.03, 0.515, 0.955)",
    
    "ease-in-cubic": "cubic-bezier(0.55, 0.055, 0.675, 0.19)",
    "ease-out-cubic": "cubic-bezier(0.215, 0.61, 0.355, 1)",
    "ease-in-out-cubic": "cubic-bezier(0.645, 0.045, 0.355, 1)",
    
    "ease-in-quart": "cubic-bezier(0.895, 0.03, 0.685, 0.22)",
    "ease-out-quart": "cubic-bezier(0.165, 0.84, 0.44, 1)",
    "ease-in-out-quart": "cubic-bezier(0.77, 0, 0.175, 1)",
    
    # Expressive
    "ease-in-back": "cubic-bezier(0.6, -0.28, 0.735, 0.045)",
    "ease-out-back": "cubic-bezier(0.175, 0.885, 0.32, 1.275)",
    "ease-in-out-back": "cubic-bezier(0.68, -0.55, 0.265, 1.55)",
}

## Layer

In [ ]:
#| export
class SVGLayer(Generatable):
    """A named SVG group (<g>) element that can contain SVG content and be styled/animated.

    SVGLayer provides a way to organize SVG content into logical groups that can be
    independently styled, positioned, and animated. Each layer is rendered as an SVG
    <g> (group) element with a data-layer attribute for CSS targeting.

    Attributes:
        name (str): Unique identifier for the layer, used for CSS targeting
        body (str): SVG content (elements) contained within this layer
        properties (dict): SVG attributes to apply to the <g> element (e.g., transform, opacity)
        pseudo_styles (dict): Pseudo-class styles (currently unused, reserved for future)
        animation (LayerAnimation): Optional animation configuration for this layer

    Examples:
        Basic layer creation:
        >>> layer = SVGLayer("background", '<rect x="0" y="0" width="100" height="100" fill="blue"/>')
        >>> print(layer.generate())
        <g data-layer="background">
        <rect x="0" y="0" width="100" height="100" fill="blue"/>
        </g>
        
        Layer with transform:
        >>> layer = SVGLayer("rotated", '<circle cx="50" cy="50" r="20"/>', 
        ...                  transform="rotate(45 50 50)")
        
        Layer with opacity:
        >>> layer = SVGLayer("faded", '<polygon points="0,0 100,0 50,100"/>',
        ...                  opacity="0.5")
        
        Modifying layer properties:
        >>> layer.set_transform("translate(10, 20)")
        >>> layer.set_opacity(0.8)
        >>> layer.hide()
        >>> layer.show()
        
        Using with SVGBuilder:
        >>> builder = SVGBuilder()
        >>> builder.adjust("terrain", '<polygon points="..." class="hills"/>')
        >>> builder.animate_layer("terrain", animation_type="fade", duration=1.0)

    Notes:
        - Layers are automatically created when using SVGBuilder.adjust()
        - The data-layer attribute enables CSS targeting: g[data-layer="name"]
        - Layers can be animated using SVGBuilder.animate_layer() or animate_cascade()
        - Use properties dict for SVG attributes like transform, opacity, visibility
        - The generate() method produces the final SVG <g> element with all attributes

    See Also:
        SVGBuilder: Main builder class that manages multiple layers
        LayerAnimation: Animation configuration for layers
        Generatable: Abstract base class for SVG elements
    """
    def __init__(self, name:str, body:str, **properties):
        self.name = name
        self.body = body
        self.properties = properties
        self.pseudo_styles = {}
        self.animation = None  # Will hold LayerAnimation

    def generate(self) -> str:
        """Generate SVG with animation support."""
    
        # Add data-layer attribute for CSS targeting
        attrs = [f'data-layer="{self.name}"']
        
        # Add existing properties
        for key, value in self.properties.items():
            attrs.append(f'{key}="{value}"')
        
        attrs_str = ' '.join(attrs)
        
        return f"<g {attrs_str}>\n{self.body}\n</g>"

    


In [ ]:
#| export
@patch
def set_transform(self: SVGLayer, transform: str):
    """Set transform attribute for the layer"""
    self.properties['transform'] = transform
    return self

@patch
def set_opacity(self: SVGLayer, opacity: float):
    """Set opacity for entire layer"""
    self.properties['opacity'] = opacity
    return self

@patch
def hide(self: SVGLayer):
    """Hide the layer"""
    self.properties['visibility'] = 'hidden'
    return self

@patch
def show(self: SVGLayer):
    """Show the layer"""
    if 'visibility' in self.properties:
        del self.properties['visibility']
    return self

## Builder

In [ ]:
#| export
class SVGBuilder(Generatable):
    """A builder class for programmatically creating SVG graphics with styles, patterns, layers, and animations.
    
    SVGBuilder provides a high-level API for constructing complex SVG documents by managing:
    - CSS styles with pseudo-classes (hover, active, etc.)
    - Reusable definitions (patterns, gradients, clip paths)
    - Named layers for organizing content
    - CSS-based animations (fade, slide, scale)
    - Text elements (titles, subtitles, legends)
    
    The builder uses a fluent interface pattern, allowing method chaining for concise code.
    
    Attributes:
        styles (dict): CSS style definitions keyed by class name
        definitions (List[Generatable]): Reusable SVG elements (patterns, gradients, etc.)
        layers (List[SVGLayer]): Named groups of SVG content
        width (int): SVG viewport width in pixels (default: 300)
        height (int): SVG viewport height in pixels (default: 300)
        blockTag (str): Root element tag name (default: "svg", can be "pattern")
        attributes (dict): Additional attributes for the root element
        title (str): Document title (default: "Untitled")
        comment (str): Optional XML comment in header
        animations (dict): Layer-name to LayerAnimation mappings
        
    Examples:
        Basic usage:
        >>> builder = SVGBuilder()
        >>> builder.width = 400
        >>> builder.height = 300
        >>> builder.adjust("background", '<rect x="0" y="0" width="400" height="300" fill="blue"/>')
        >>> svg_xml = builder.xml()
        
        With styles and patterns:
        >>> loader = SVGPatternLoader(patterns_dir="patterns")
        >>> pattern, path = loader.stylizedPattern('ball.svg', patName="water")
        >>> builder.add_definition(pattern)
        >>> water_style = StyleCSS("water", fill=path, stroke="#0D47A1")
        >>> builder.add_style(water_style)
        >>> builder.adjust("lake", '<polygon points="..." class="water"/>')
        
        With animations:
        >>> builder.animate_layer("background", animation_type="fade", duration=1.0)
        >>> builder.animate_cascade(duration=0.8, stagger=0.2)
        
        Method chaining:
        >>> (builder
        ...     .add_title("My Map", subtitle="Interactive SVG")
        ...     .animate_cascade(duration=1.0, stagger=0.3)
        ...     .show())
        
    
    
    See Also:
        StyleCSS: CSS style management with pseudo-classes
        SVGLayer: Named layer/group management
        SVGPatternLoader: Loading SVG files as reusable patterns
        LayerAnimation: Animation configuration
        
    Notes:
        - Use adjust() to add/update layers by name
        - Use add_definition() for reusable elements (avoids duplication by ID)
        - Use add_style() for CSS classes
        - Call xml() for complete XML document or generate() for just the SVG element
        - Use show() in Jupyter/solveit for interactive preview
    """

    def __init__(self):
        self.styles = {}
        self.definitions: List[Generatable] = []
        self.layers: List[SVGLayer] = [SVGLayer("root", "")]
        self.width = 300
        self.height = 300
        self.blockTag = "svg"
        self.attributes = {} 
        self.title = "Untitled"
        self.comment = None

        # NEW: Animation support
        self.animations = {}  # layer_name -> LayerAnimation
        self.animation_counter = 0  # For unique animation IDs

    @property
    def body(self):
        if len(self.layers) == 1:
            return self.layers[0].body
        
        return "\n".join([x.generate() for x in self.layers])


    def generate(self)->str:
        return "this is patched below"

    def add_definition(self, item: Generatable):
        if not isinstance(item, Generatable):
            raise TypeError(f"Item must be Generatable, got {type(item)}")
        
        # Check for duplicate by id if the item has one
        item_id = getattr(item, 'id', None)
        if item_id is not None:
            for existing in self.definitions:
                if getattr(existing, 'id', None) == item_id:
                    return  # Already exists, skip
        
        self.definitions.append(item)


    def add_style(self, item: StyleCSS):
        if not isinstance(item, StyleCSS):
            raise TypeError(f"Item must be StyleCSS, got {type(item)}")
        self.styles[item.name] = item

    def xml(self):
        ret = "<?xml version='1.0' encoding='utf-8'?>\n"
        ret += self.generate()
        return ret

   


We need to modify add_definition so it adds unique definitions based on the id property

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's the updated `add_definition` method that checks for uniqueness based on the `id` property:

```python
def add_definition(self, item: Generatable):
    if not isinstance(item, Generatable):
        raise TypeError(f"Item must be Generatable, got {type(item)}")
    
    # Check for duplicate by id if the item has one
    item_id = getattr(item, 'id', None)
    if item_id is not None:
        for existing in self.definitions:
            if getattr(existing, 'id', None) == item_id:
                return  # Already exists, skip
    
    self.definitions.append(item)
```

This will:
1. Get the `id` attribute from the item (if it exists)
2. Check if any existing definition has the same `id`
3. Only add the item if no duplicate is found

If you want to return a boolean indicating whether it was added, or return the existing item when a duplicate is found:

```python
def add_definition(self, item: Generatable):
    """Add a definition, skipping duplicates based on id. Returns the item (existing or new)."""
    if not isinstance(item, Generatable):
        raise TypeError(f"Item must be Generatable, got {type(item)}")
    
    item_id = getattr(item, 'id', None)
    if item_id is not None:
        for existing in self.definitions:
            if getattr(existing, 'id', None) == item_id:
                return existing  # Return existing duplicate
    
    self.definitions.append(item)
    return item
```

In [ ]:
#| export
@patch
def updateLayers(self:SVGBuilder,layers:[str]):
    for index, layer in enumerate(layers):
        if index < len(self.layers):
            self.layers[index].body = layer
        else:
            aLayer = SVGLayer(f"layer{index}",layer)
            self.layers.append(aLayer)

@patch       
def adjust(self:SVGBuilder,name:str,body:str):
    for index, layer in enumerate(self.layers):
        if layer.name == name:
            self.layers[index].body = body
            return
    
    aLayer = SVGLayer(name,body)
    self.layers.append(aLayer)


In [ ]:
#| export
@patch
def _header(self: SVGBuilder) -> str:
     
    ret = f"<title> {self.title} </title>\n"
    if self.comment is not None:
        ret += f"<!-- \n{self.comment}\n -->\n"
    
    
    # Add definitions
    if len(self.definitions) > 0:
        defs_content = "\n".join([x.generate() for x in self.definitions])
        ret += f"  <defs>\n    {defs_content}\n  </defs>\n"
    
    # Add styles (including animations)
    style_content = ""
    
    # Regular CSS styles
    if len(self.styles) > 0:
        keys = sorted(self.styles.keys(), reverse=True)
        myStyles = [self.styles[key] for key in keys]
        style_content += StyleCSS.generate(myStyles) + "\n"
    
    # Animation CSS
    animation_css = self._generate_animation_css()
    if animation_css:
        style_content += animation_css + "\n"
    
    if style_content:
        ret += f"  <style>\n{style_content}  </style>\n"

    return ret

@patch
def generate(self: SVGBuilder) -> str:
    attrs = ' '.join(f'{k}="{v}"' for k, v in self.attributes.items())
    ret = f'<{self.blockTag} {attrs} width="{self.width}" height="{self.height}" '
    ret += f'viewBox="0 0 {self.width} {self.height}" xmlns="http://www.w3.org/2000/svg">\n'

    ret += self._header()

    # Add body
    ret += "\n"
    ret += self.body
    ret += f"\n</{self.blockTag}>"
    
    return ret

In [ ]:
#| export
@patch
def get_easing(self: SVGBuilder, easing_name: str) -> str:
    """Get easing function, with preset support."""
    return EASING_PRESETS.get(easing_name, easing_name)

In [ ]:
#| export
@patch
def _generate_animation_css(self: SVGBuilder) -> str:
    """Generate CSS animations for all animated layers."""
    if not self.animations:
        return ""
    
    css_parts = []
    
    for layer_name, anim in self.animations.items():
        anim_id = f"anim_{layer_name}_{self.animation_counter}"
        self.animation_counter += 1
        
        # Generate keyframes
        keyframes = self._generate_keyframes(anim, anim_id)
        
        # Generate animation rule for layer
        animation_rule = self._generate_animation_rule(layer_name, anim, anim_id)
        
        css_parts.append(keyframes)
        css_parts.append(animation_rule)
    
    return "\n".join(css_parts)

@patch
def _generate_keyframes(self: SVGBuilder, anim: LayerAnimation, anim_id: str) -> str:
    """Generate @keyframes for an animation."""
    
    if anim.animation_type == "fade":
        return f"""
@keyframes {anim_id} {{
    from {{ opacity: {anim.start_opacity}; }}
    to {{ opacity: {anim.end_opacity}; }}
}}"""
    
    elif anim.animation_type == "slide":
        x_offset, y_offset = anim.slide_from or (0, -50)
        return f"""
@keyframes {anim_id} {{
    from {{ 
        opacity: {anim.start_opacity};
        transform: translate({x_offset}px, {y_offset}px);
    }}
    to {{ 
        opacity: {anim.end_opacity};
        transform: translate(0, 0);
    }}
}}"""
    
    elif anim.animation_type == "scale":
        return f"""
@keyframes {anim_id} {{
    from {{ 
        opacity: {anim.start_opacity};
        transform: scale({anim.start_scale});
    }}
    to {{ 
        opacity: {anim.end_opacity};
        transform: scale({anim.end_scale});
    }}
}}"""
    
    return ""

@patch
def _generate_animation_rule(self: SVGBuilder, layer_name: str, 
                             anim: LayerAnimation, anim_id: str) -> str:
    """Generate CSS rule that applies animation to layer."""
    
    # CSS selector for the layer's <g> element
    selector = f'g[data-layer="{layer_name}"]'
    
    return f"""
{selector} {{
    animation-name: {anim_id};
    animation-duration: {anim.duration}s;
    animation-delay: {anim.delay}s;
    animation-timing-function: {anim.easing};
    animation-fill-mode: {anim.fill_mode};
    animation-iteration-count: {anim.iteration_count};
    transform-origin: {anim.transform_origin};
    
    /* Start in initial state */
    opacity: {anim.start_opacity};
}}"""

In [ ]:
#| export
@patch
def animate_group(self: SVGBuilder,
                 layer_names: list[str],
                 duration: float = 1.0,
                 delay: float = 0.0,
                 animation_type: str = "fade",
                 easing: str = "ease-in-out") -> SVGBuilder:
    """Animate multiple layers simultaneously.
    
    Args:
        layer_names: List of layer names to animate together
        duration: Duration for all layers
        delay: Delay before group starts
        animation_type: Animation type for all
        easing: Easing for all
        
    Example:
        # Fade in background and water together
        builder.animate_group(["background", "water"], delay=0.5)
    """
    for name in layer_names:
        self.animate_layer(
            name,
            animation_type=animation_type,
            duration=duration,
            delay=delay,
            easing=easing
        )
    
    return self

In [ ]:
#| export
@patch
def animate_layer_full_cycle(self: SVGBuilder,
                             layer_name: str,
                             fade_in_duration: float,
                             hold_duration: float,
                             fade_out_duration: float,
                             delay: float = 0.0,
                             easing: str = "ease-in-out",
                             loop: bool = False) -> SVGBuilder:
    """Animate a layer through fade in -> hold -> fade out cycle.
    
    Args:
        layer_name: Name of the layer to animate
        fade_in_duration: Duration of fade in (seconds)
        hold_duration: Duration at full opacity (seconds)
        fade_out_duration: Duration of fade out (seconds)
        delay: Delay before animation starts (seconds)
        easing: CSS easing function
        loop: Whether to loop infinitely
        
    Returns:
        self for chaining
        
    Example:
        builder.animate_layer_full_cycle("terrain", 
                                         fade_in_duration=1.0,
                                         hold_duration=2.0, 
                                         fade_out_duration=1.0,
                                         delay=0.5)
    """
    
    total = fade_in_duration + hold_duration + fade_out_duration
    
    # Handle edge case where fade_out is 0 (last layer)
    if fade_out_duration == 0:
        total = fade_in_duration + hold_duration
    
    # Calculate keyframe percentages
    fade_in_pct = (fade_in_duration / total) * 100
    hold_pct = ((fade_in_duration + hold_duration) / total) * 100
    
    anim_id = f"cycle_{layer_name}_{self.animation_counter}"
    self.animation_counter += 1
    
    # Generate custom keyframes
    if fade_out_duration == 0:
        # No fade out - stay at opacity 1
        keyframes = f"""
@keyframes {anim_id} {{
    0% {{ opacity: 0; }}
    {fade_in_pct:.1f}% {{ opacity: 1; }}
    100% {{ opacity: 1; }}
}}"""
    else:
        # Full cycle with fade out
        keyframes = f"""
@keyframes {anim_id} {{
    0% {{ opacity: 0; }}
    {fade_in_pct:.1f}% {{ opacity: 1; }}
    {hold_pct:.1f}% {{ opacity: 1; }}
    100% {{ opacity: 0; }}
}}"""
    
    # Generate animation rule
    selector = f'g[data-layer="{layer_name}"]'
    iteration = "infinite" if loop else "1"
    easing_func = self.get_easing(easing)
    
    rule = f"""
{selector} {{
    animation-name: {anim_id};
    animation-duration: {total}s;
    animation-delay: {delay}s;
    animation-timing-function: {easing_func};
    animation-fill-mode: forwards;
    animation-iteration-count: {iteration};
    opacity: 0;
}}

{keyframes}"""
    
    # Store in custom animations
    if not hasattr(self, 'custom_animations'):
        self.custom_animations = []
    self.custom_animations.append(rule)
    
    return self

@patch
def _generate_animation_css(self: SVGBuilder) -> str:
    """Generate CSS animations for all animated layers."""
    if not self.animations and not hasattr(self, 'custom_animations'):
        return ""
    
    css_parts = []
    
    # Original single-animation layers
    for layer_name, anim in self.animations.items():
        anim_id = f"anim_{layer_name}_{self.animation_counter}"
        self.animation_counter += 1
        
        # Generate keyframes
        keyframes = self._generate_keyframes(anim, anim_id)
        
        # Generate animation rule for layer
        animation_rule = self._generate_animation_rule(layer_name, anim, anim_id)
        
        css_parts.append(keyframes)
        css_parts.append(animation_rule)
    
    # Custom full-cycle animations
    if hasattr(self, 'custom_animations'):
        css_parts.extend(self.custom_animations)
    
    return "\n".join(css_parts)


In [ ]:
#| export
@patch
def to_icon(self: SVGBuilder, size: int = 50) -> str:
    """Wrap SVG in fixed-size container with aspect ratio preservation."""
    content = self.generate()
    viewBox = f"0 0 {self.width} {self.height}"
    
    return f'''<?xml version='1.0' encoding='utf-8'?>
<svg xmlns="http://www.w3.org/2000/svg" 
     width="{size}" 
     height="{size}"
     viewBox="{viewBox}"
     preserveAspectRatio="xMidYMid meet">
{content}
</svg>'''

### Common graph elements

In [ ]:
svgExample = """
<?xml version='1.0' encoding='utf-8'?><svg width="300" height="300" viewBox="0 0 300 300" xmlns="http://www.w3.org/2000/svg">

<defs>
<pattern id="pattern-water-ball" patternUnits="userSpaceOnUse" width="40.0" height="40.0" patternTransform="scale(0.5)">
   <style>


.Yellow_green_hills {
   fill: #D4E157;
   stroke: #D4E157;
   stroke-width: 1.5;
   stroke-opacity: 0.9;
}
</style>
   
    <rect xmlns="http://www.w3.org/2000/svg" id="Artboard1" x="0" y="0" width="40" height="40" class="Yellow_green_hills" />
     <rect x="0" y="0" width="40" height="40" fill="#B3E5FC" />
    <g xmlns="http://www.w3.org/2000/svg" xmlns:ns1="http://www.serif.com/" id="Artboard11" ns1:id="Artboard1">
        <ellipse cx="20.321" cy="19.995" rx="13.786" ry="14.887" class="Yellow_green_hills" />
    </g>

    </pattern>
</defs><style>.water{
   fill:url(#pattern-water-ball);
   stroke:#0D47A1;
   stroke-width:2;
   stroke-opacity:1.0;
}

 .water{
   fill:url(#pattern-water-ball);
   stroke:#0D47A1;
   stroke-width:2;
   stroke-opacity:1.0;
}

 .water{
   fill:url(#pattern-water-ball);
   stroke:#0D47A1;
   stroke-width:2;
   stroke-opacity:1.0;
}
</style>
<polygon points="276,73 276,227 150,300 24,227 24,73 150,0 " class="Rich_lowland_green"/>
<polygon points="268,78 268,222 150,290 32,222 32,78 150,10 " class="Yellow_green_hills"/>
<polygon points="259,83 259,217 150,280 41,217 41,83 150,20 " class="water"/>
</svg>
"""

In [ ]:
aBuilder = SVGBuilder()
hoover = StyleCSS("hoover",fill="#007fff",cursor= "pointer")

for x in StyleCSS.elevations()[:3]:
    x.customize(hoover)
    aBuilder.add_style(x)

print(aBuilder.xml())
#aBuilder.styles, hills.name


In [ ]:
#| export
@patch
def add_text_layer(self: SVGBuilder, name: str, text: str, x: float = None, y: float = None, 
                    class_name: str = None, text_anchor: str = "middle", **properties):
    """Add a text element to a layer with optional styling"""
    if x is None:
        x = self.width / 2
    if y is None:
        y = self.height / 2
    
    attrs = [f'x="{x}"', f'y="{y}"', f'text-anchor="{text_anchor}"']
    
    if class_name:
        attrs.append(f'class="{class_name}"')
    
    for key, value in properties.items():
        attrs.append(f'{key.replace("_", "-")}="{value}"')
    
    body = f'<text {" ".join(attrs)}>{text}</text>'
    self.adjust(name, body)
    return self

@patch
def add_title(self: SVGBuilder, title: str, subtitle: str = None, 
              title_class: str = "title", subtitle_class: str = "subtitle",
              title_offset: float = -20, subtitle_offset: float = 20):
    """Add centered title and optional subtitle to the SVG"""
    center_x = self.width / 2
    center_y = self.height / 2
    
    title_body = f'<text x="{center_x}" y="{center_y + title_offset}" text-anchor="middle" class="{title_class}">{title}</text>'
    
    if subtitle:
        subtitle_body = f'<text x="{center_x}" y="{center_y + subtitle_offset}" text-anchor="middle" class="{subtitle_class}">{subtitle}</text>'
        body = title_body + '\n' + subtitle_body
    else:
        body = title_body
    
    self.adjust("titles", body)
    return self

@patch  
def add_centered_text(self: SVGBuilder, text: str, y_offset: float = 0, class_name: str = "centered-text"):
    """Add a single centered text element"""
    center_x = self.width / 2
    center_y = self.height / 2 + y_offset
    body = f'<text x="{center_x}" y="{center_y}" text-anchor="middle" class="{class_name}">{text}</text>'
    self.adjust("centered_text", body)
    return self

In [ ]:
#| export
@patch
def legendOverlay(self: SVGBuilder,
    styles:[StyleCSS], #the list of styles we want to have as our legend. Style name will be the text
    bg_fill="white", # the default background color
    xOffset=10, # how far over we want
    yOffset=10 #and in the y direction
    ):

    ret = ""
    
    radius = 15
    pad = 10
    x = radius + pad + xOffset
    y = radius + pad + yOffset
    
    # Calculate total height needed
    total_height = len(styles) * (radius * 2 + pad) + pad 
    total_width = 150 + x # Adjust as needed for text
    
    # Add background rectangle
    if bg_fill:
        ret += f'\t<rect x="{xOffset}" y="{yOffset}" width="{total_width}" height="{total_height}" fill="{bg_fill}" rx="5"/>\n'
    
    for style in styles:
        name = " ".join(style.name.split("_"))
        ret += f'\t<circle cx="{x}" cy="{y}" r="{radius}" class="{style.name}"/>\n'
        ret += f'\t<text x="{x + pad + radius}" y="{y}" text-anchor="start" dy="0.3em" class="keyLabel">{name}</text>\n'
        y += radius * 2 + pad

    return ret

NameError: name 'patch' is not defined

## Drawing

In [ ]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

In [ ]:
#| export
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', rel='stylesheet', type='text/css'),
)

In [ ]:
#| export
app, rt = fast_app(hdrs=daisy_hdrs)
#rt = app.route

In [ ]:
?fast_app

In [ ]:
#| export
def get_preview(app):
    return partial(HTMX, app=app, host=None, port=5002)
preview = get_preview(app)

In [ ]:
preview(P("Hello World"))

In [ ]:
#| export
@patch
def show(self:SVGBuilder,dim=None):
    if dim is None:
        return preview(Div(NotStr(self.xml())))
    return preview(Div(NotStr(self.to_icon(dim))))

### Patterns

In [ ]:
#| export
class SVGPatternLoader:
    """Loads and converts SVG files to pattern definitions"""
    
    def __init__(self, patterns_dir: str = None):
        if patterns_dir is None:
            patterns_dir = Path(__file__).parent / "patterns"
        self.patterns_dir = Path(patterns_dir)
        self._pattern_cache = {}
    

    
    def get_available_patterns(self) -> list:
        """Return list of available SVG pattern files"""
        return [f.name for f in self.patterns_dir.glob('*.svg')]

In [ ]:
aLoader = SVGPatternLoader(patterns_dir="patterns")
aLoader.get_available_patterns()

In [ ]:
#| export

@patch
def _extract_styles(self:SVGPatternLoader, element, style_map, existing_styles, counter):
    """Recursively extract styles from element and its children"""
    
    # Check if this element has a style attribute
    if 'style' in element.attrib:
        style_str = element.attrib['style']
        style_obj, is_new = StyleCSS.from_style_string(style_str, counter, existing_styles)
        
        # Map this element to its style class name
        style_map[id(element)] = style_obj.name
        
        # Add to existing_styles if new
        if is_new:
            existing_styles.append(style_obj)
            counter += 1
    
    # Recurse on all children
    for child in element:
        counter = self._extract_styles(child, style_map, existing_styles, counter)
    
    return counter



@patch
def find_css(self:SVGPatternLoader, filename: str) -> [StyleCSS]:
    filepath = self.patterns_dir / filename
    tree = ET.parse(filepath)
    root = tree.getroot()
    
    # Initialize tracking
    style_map = {}
    existing_styles = []
    counter = 0
    
    # First pass: extract all styles recursively
    for child in root:
        counter = self._extract_styles(child, style_map, existing_styles, counter)
    
    # Now existing_styles has all your StyleCSS objects
    return existing_styles


In [ ]:
aLoader = SVGPatternLoader(patterns_dir="data/patterns")
print(StyleCSS.generate(aLoader.find_css('eye.svg')))

In [ ]:
#| export

@patch
def _get_dimensions(self:SVGPatternLoader, svg_elem):
    """Get dimensions, preferring viewBox over width/height"""
    viewbox = svg_elem.get('viewBox')
    if viewbox:
        parts = viewbox.split()
        if len(parts) == 4:
            return float(parts[2]), float(parts[3])
    
    # Fallback to width/height with parsing
    width_str = svg_elem.get('width', '100')
    height_str = svg_elem.get('height', '100')
    
    # Parse dimensions, handling %, px, pt
    def parse_dim(val, default=100):
        val = str(val).strip()
        if val.endswith('%'):
            return default
        val = val.replace('pt', '').replace('px', '')
        try:
            return float(val)
        except ValueError:
            return default
    
    return parse_dim(width_str), parse_dim(height_str)



#| export
@patch
def load_pattern(self:SVGPatternLoader, filename: str, pattern_id: str, scale: float = 1.0, prefix="pat") -> SVGBuilder:
    """Load an SVG pattern from file and return configured SVGBuilder"""
    filepath = self.patterns_dir / filename
    
    with open(filepath, 'r') as f:
        content = f.read()
    
    soup = BeautifulSoup(content, 'xml')
    svg = soup.find('svg')
    
    # Extract dimensions using helper method
    width, height = self._get_dimensions(svg)
    
    builder = SVGBuilder()
    builder.blockTag = "pattern"
    builder.width = width
    builder.height = height
    builder.attributes = {
        'id': pattern_id,
        'patternUnits': 'userSpaceOnUse',
        'patternTransform': f'scale({scale})'
    }
    
    # Extract styles and create class mappings
    styles = {}
    style_to_class = {}
    
    # Elements to extract
    element_types = ['path', 'polygon', 'ellipse', 'rect', 'circle', 'line', 'polyline']
    
    for elem in svg.find_all(element_types):
        style = elem.get('style', '')
        if style and style not in style_to_class:
            class_idx = len(styles)
            class_name = f"{prefix}_{class_idx}"
            style_obj, _ = StyleCSS.from_style_string(style, class_idx)
            style_obj.name = class_name  # <-- ADD THIS LINE
            styles[class_name] = style_obj
            style_to_class[style] = class_name
    
    # Extract elements and replace inline styles with classes
    elements = []
    for elem in svg.find_all(element_types):
        style = elem.get('style', '')
        if style:
            elem['class'] = style_to_class[style]
            del elem['style']
        elements.append(str(elem))
    
    # Add styles to builder
    for name, style_obj in styles.items():
        builder.add_style(style_obj)
    
    # Combine all elements into body
    builder.updateLayers(['\n'.join(elements)])
    
    
    return builder


In [ ]:
#| export
@patch
def load_pattern(self: SVGPatternLoader, 
                filename: str, 
                pattern_id: str, 
                scale: float = 1.0, 
                prefix: str = None) -> SVGBuilder:
    """Load an SVG pattern from file and return configured SVGBuilder"""
    
    if prefix is None:
        prefix = pattern_id
        
    filepath = self.patterns_dir / filename
    
    with open(filepath, 'r') as f:
        content = f.read()
    
    soup = BeautifulSoup(content, 'xml')
    svg = soup.find('svg')
    
    width, height = self._get_dimensions(svg)
    
    builder = SVGBuilder()
    builder.blockTag = "pattern"
    builder.width = width
    builder.height = height
    builder.attributes = {
        'id': pattern_id,
        'patternUnits': 'userSpaceOnUse',
        'patternTransform': f'scale({scale})'
    }
    
    element_types = ['path', 'polygon', 'ellipse', 'rect', 'circle', 'line', 'polyline']
    
    # Assign a single class to ALL graphic elements
    class_name = f"{prefix}_path"
    
    # Create a default style (will be overridden by user)
    default_style = StyleCSS(class_name, fill="#000000", stroke="none")
    builder.add_style(default_style)
    
    # Extract elements and ADD class to each
    elements = []
    for elem in svg.find_all(element_types):
        elem['class'] = class_name  # Always add the class
        # Optionally remove inline style if present
        if elem.get('style'):
            del elem['style']
        elements.append(str(elem))
    
    builder.updateLayers(['\n'.join(elements)])
    
    return builder


In [ ]:
aLoader = SVGPatternLoader(patterns_dir="data/patterns")
aPat = aLoader.load_pattern('small.svg',"water")


In [ ]:
aPat.blockTag = "SVG"
aPat.attributes = {} 
aPat.add_centered_text("Scroll Down", y_offset= -490)
#print(aPat.generate())
aPat.show(100)

### Patterns with Builders

In [ ]:
#| export
@patch
def stylizedPattern(self:SVGPatternLoader, name,colors= StyleCSS.elevations(),patName="egg",transform="scale(1)"):
    
    aPat = self.load_pattern(name,patName,prefix=f"pat_{patName}")
   
    for i, x in enumerate(colors):
        aStyle = StyleCSS(f"pat_{patName}_{i}")
        for x in colors[i].properties.keys():
            aStyle.properties[x] = colors[i].properties[x]
        aPat.add_style(aStyle)

    aPat.attributes['patternTransform'] = transform

    return aPat, f"url(#{patName})"


In [ ]:
@patch
def demoStylize(self:StyleDemo):

    aLoader = SVGPatternLoader(patterns_dir="data/patterns")
    myPattern, path = aLoader.stylizedPattern('ball.svg')
    bMaker = SVGBuilder()
    bMaker.add_definition(myPattern)
    bStyle = StyleCSS("water",fill=path)
    bMaker.add_style(bStyle)

    body = """<polygon points="259,83 259,217 150,280 41,217 41,83 150,20 " class="water"/>"""
    bMaker.updateLayers([body])
    
    return bMaker.show()

StyleDemo().demoStylize()

In [ ]:
#| export
simpleSVG = """
<rect class="bacon_0" height="40" id="Artboard1" width="40" x="0" y="0"/>
<rect class="bacon_1" height="40" id="Artboard1" width="40" x="40" y="0"/>
<rect class="bacon_2" height="40" id="Artboard1" width="40" x="0" y="40"/>
<rect class="bacon_3" height="40" id="Artboard1" width="40" x="40" y="40"/>
"""

@patch
def demoDrawColor(self:StyleDemo,prefix="bacon", body = simpleSVG):
    
    dummy = StyleCSS("blank")

    patterns = []
    styles = []
    levels = StyleCSS.elevations()
    aLoader = SVGPatternLoader(patterns_dir="data/patterns")
   
    for i, level in enumerate(levels):
        if i < len(levels) - 1:
            someColors = [levels[i],levels[i+1]]
            #print(", ".join([x.properties["fill"] for x in someColors]))
            patternName = f"{prefix}_{i}"
            myPattern, path = aLoader.stylizedPattern('ball.svg',colors=someColors,patName=patternName)
            patterns.append(myPattern)
            styles.append(StyleCSS(patternName,fill=path))
        

    retBuilder = SVGBuilder()
    retBuilder.updateLayers([body])
    

    for x in patterns:
        retBuilder.add_definition(x)

    for x in styles:
        retBuilder.add_style(x)

    file_name = "junk/style_{prefix}_demo.svg"
    with open(file_name, 'w') as file_object:
        file_object.write(retBuilder.xml())
    return retBuilder

#aBuilder = StyleDemo().demoDrawColor()
#print(aBuilder.xml())

In [ ]:
@patch
def demoOverlay(self:StyleDemo):

    aLoader = SVGPatternLoader(patterns_dir="data/patterns")

    patterns = []
    styles = []
    levels = StyleCSS.elevations()[:3]

    clearBlackStyle = StyleCSS("clearBlackStyle",fill="none",stroke="#000000")
    clearBlueStyle = StyleCSS("clearBlueStyle",fill="none",stroke="#000000ff", stroke_width=10)
    someColors = [clearBlueStyle,clearBlackStyle]

    lime = StyleCSS("lime",fill="#e6d74fd7",stroke="#d4d123ae",fill_opacity=0.4)

    prefix = "overlay"
    fileName = 'jungle1.svg' # 'ball.svg'
    i = 5
    patternName = f"{prefix}_{i}"
    myPattern, path = aLoader.stylizedPattern(fileName,
    colors=someColors,
    patName=patternName,
    transform="scale(0.25)")

    patterns.append(myPattern)
    styles.append(StyleCSS(patternName,fill=path))

    bMaker = SVGBuilder()
    bMaker.add_definition(myPattern)

    bStyle = StyleCSS("water",fill=path)

    bMaker.add_style(bStyle)
    bMaker.add_style(lime)

    body = """
     <polygon points="259,83 259,217 150,280 41,217 41,83 150,20 " class="water"/>
    <polygon points="259,83 259,217 150,280 41,217 41,83 150,20 " class="lime"/>
   

    """
    bMaker.updateLayers([body])
    file_name = "tmp/style_{prefix}_demo.svg"
    with open(file_name, 'w') as file_object:
        file_object.write(bMaker.xml())

    return bMaker.show()
    return bMaker.xml()
StyleDemo().demoOverlay()
#print(StyleDemo().demoOverlay())

I want to do something like legend where I add a title and a subtitle

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I can help you create a legend with a title and subtitle for your SVG! Based on your code, you're working with SVG patterns and styles. Here's how you could add a legend component:

```python
@patch
def add_legend(self:SVGBuilder, title, subtitle, items, x=10, y=10):
    """
    Add a legend with title, subtitle, and pattern/color samples
    
    items: list of dicts like [{'label': 'Water', 'class': 'water'}, ...]
    """
    legend_parts = []
    current_y = y
    
    # Title
    legend_parts.append(f'<text x="{x}" y="{current_y}" style="font-size:18px;font-weight:bold;fill:#333">{title}</text>')
    current_y += 20
    
    # Subtitle
    legend_parts.append(f'<text x="{x}" y="{current_y}" style="font-size:12px;fill:#666">{subtitle}</text>')
    current_y += 25
    
    # Legend items
    for item in items:
        # Sample rectangle
        legend_parts.append(f'<rect x="{x}" y="{current_y-10}" width="20" height="20" class="{item["class"]}"/>')
        # Label
        legend_parts.append(f'<text x="{x+25}" y="{current_y+5}" style="font-size:14px;fill:#333">{item["label"]}</text>')
        current_y += 30
    
    return '\n'.join(legend_parts)
```

Here's an example of how to use it with your existing code:

```python
@patch
def demoWithLegend(self:StyleDemo):
    aLoader = SVGPatternLoader(patterns_dir="patterns")
    myPattern, path = aLoader.stylizedPattern('ball.svg')
    
    bMaker = SVGBuilder()
    bMaker.add_definition(myPattern)
    bStyle = StyleCSS("water", fill=path)
    bMaker.add_style(bStyle)

    body = """<polygon points="259,83 259,217 150,280 41,217 41,83 150,20 " class="water"/>"""
    
    # Add legend
    legend_items = [
        {'label': 'Water Pattern', 'class': 'water'}
    ]
    legend = bMaker.add_legend("Pattern Demo", "Stylized SVG Patterns", legend_items, x=300, y=50)
    
    bMaker.updateLayers([body, legend])
    
    return bMaker.show()

StyleDemo().demoWithLegend()
```

Would you like me to adjust the styling, positioning, or add more features to the legend?

I need something simpler that would just put a title and subtitle centered at the top

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Whoops! An error (E300) occurred while processing your request.
If this problem persists, please contact us on Discord.
Please include your dialog url and error code in your message.

In [ ]:
#| export
@patch
def get_layer(self: SVGBuilder, name: str) -> SVGLayer:
    """Get layer by name"""
    for layer in self.layers:
        if layer.name == name:
            return layer
    return None

@patch
def remove_layer(self: SVGBuilder, name: str):
    """Remove a layer by name"""
    self.layers = [layer for layer in self.layers if layer.name != name]
    return self

@patch
def clear_layers(self: SVGBuilder):
    """Clear all layers except root"""
    self.layers = [SVGLayer("root", "")]
    return self

In [ ]:
#| export
@patch
def animate_layer(self: SVGBuilder, 
                  layer_name: str,
                  animation_type: str = "fade",
                  duration: float = 1.0,
                  delay: float = 0.0,
                  easing: str = "ease-in-out",
                  **kwargs) -> SVGBuilder:
    """Add animation to a specific layer.
    
    Args:
        layer_name: Name of the layer to animate
        animation_type: "fade", "slide", "scale", or "none"
        duration: Animation duration in seconds
        delay: Delay before animation starts in seconds
        easing: CSS easing function (ease, ease-in, ease-out, ease-in-out, 
                linear, or cubic-bezier)
        **kwargs: Additional animation parameters
        
    Returns:
        self for chaining
        
    Example:
        builder.animate_layer("terrain", "fade", duration=1.5, delay=0.5)
    """
    animation = LayerAnimation(
        animation_type=animation_type,
        duration=duration,
        delay=delay,
        easing=easing,
        **kwargs
    )
    
    self.animations[layer_name] = animation
    
    # Find the layer and attach animation
    for layer in self.layers:
        if layer.name == layer_name:
            layer.animation = animation
            break
    
    return self

In [ ]:
#| export
@patch
def animate_cascade(self: SVGBuilder,
                   duration: float = 1.0,
                   stagger: float = 0.3,
                   animation_type: str = "fade",
                   easing: str = "ease-in-out") -> SVGBuilder:
    """Animate all layers in sequence with staggered timing.
    
    Args:
        duration: Duration of each layer's animation
        stagger: Delay between each layer's start time
        animation_type: Type of animation for all layers
        easing: Easing function for all layers
        
    Example:
        # Each layer fades in 0.3s after the previous
        builder.animate_cascade(duration=1.0, stagger=0.3)
    """
    for i, layer in enumerate(self.layers):
        if layer.name != "root":  # Skip root
            delay = i * stagger
            self.animate_layer(
                layer.name,
                animation_type=animation_type,
                duration=duration,
                delay=delay,
                easing=easing
            )
    
    return self

### as a script

I would love an example that shows off some of the features of this in a main function which would be execute if the the file is called directly. I think the 
`
def main():
    """The main entry point for the script when run directly."""
    greet("Alice")
    greet("Bob")

# This code block only runs when the script is executed as the main program
if __name__ == "__main__":
    main()
`
kind of pattern is what I am looking for

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Here's a comprehensive example that showcases the main features of your SVG builder system:

```python
def main():
    """Demo showcasing SVGBuilder features: patterns, styles, layers, and animations"""
    
    # Initialize
    aLoader = SVGPatternLoader(patterns_dir="patterns")
    builder = SVGBuilder()
    builder.width = 400
    builder.height = 500
    builder.title = "SVG Builder Feature Demo"
    
    # 1. Create a patterned background
    bg_pattern, bg_path = aLoader.stylizedPattern(
        'ball.svg',
        colors=StyleCSS.elevations()[:2],
        patName="background",
        transform="scale(0.3)"
    )
    builder.add_definition(bg_pattern)
    bg_style = StyleCSS("bg_layer", fill=bg_path, opacity=0.3)
    builder.add_style(bg_style)
    
    # 2. Add background layer
    bg_body = f'<rect x="0" y="0" width="{builder.width}" height="{builder.height}" class="bg_layer"/>'
    builder.adjust("background", bg_body)
    
    # 3. Create main content with hover effect
    hills = StyleCSS("hills", fill="#D4E157", stroke="#333333", stroke_width=2, opacity=0.9)
    hover = StyleCSS("hover_effect", fill="#007fff", cursor="pointer")
    hills.customize(hover)
    builder.add_style(hills)
    
    # 4. Add geometric shape
    shape_body = '<polygon points="200,150 350,200 300,300 100,300 50,200" class="hills"/>'
    builder.adjust("shape", shape_body)
    
    # 5. Add title and subtitle
    builder.add_title(
        "SVG Builder Demo",
        subtitle="Patterns • Styles • Animations",
        title_offset=-180,
        subtitle_offset=-155
    )
    
    # Add title/subtitle styles
    title_style = StyleCSS("title", 
                          fill="#2c3e50",
                          font_size="24px",
                          font_weight="bold",
                          font_family="Arial, sans-serif")
    subtitle_style = StyleCSS("subtitle",
                             fill="#7f8c8d", 
                             font_size="14px",
                             font_family="Arial, sans-serif")
    builder.add_style(title_style)
    builder.add_style(subtitle_style)
    
    # 6. Add legend
    legend_styles = StyleCSS.elevations()[:4]
    legend_body = builder.legend(legend_styles, xOffset=20, yOffset=350)
    builder.adjust("legend", legend_body)
    
    # Add legend label style
    key_style = StyleCSS("keyLabel",
                        fill="#333",
                        font_size="12px",
                        font_family="Arial, sans-serif")
    builder.add_style(key_style)
    
    # 7. Animate layers with cascade effect
    builder.animate_cascade(
        duration=0.8,
        stagger=0.2,
        animation_type="fade",
        easing="ease-out"
    )
    
    # 8. Save and display
    output_file = "junk/feature_demo.svg"
    with open(output_file, 'w') as f:
        f.write(builder.xml())
    
    print(f"✓ Generated: {output_file}")
    print(f"✓ Dimensions: {builder.width}x{builder.height}")
    print(f"✓ Layers: {len(builder.layers)}")
    print(f"✓ Styles: {len(builder.styles)}")
    print(f"✓ Animations: {len(builder.animations)}")
    
    return builder.show()

if __name__ == "__main__":
    main()
```

This example demonstrates:

1. **Pattern loading** - Loading and styling SVG patterns
2. **Layer management** - Multiple named layers
3. **Style creation** - CSS styles with hover effects
4. **Text elements** - Titles and subtitles
5. **Legend generation** - Automatic legend with color swatches
6. **Animations** - Cascading fade-in animations
7. **File output** - Saving to SVG file

The script will only run when executed directly, not when imported as a module!

can you generate some good docstring for SVGBuilder and some of its short comings

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's a comprehensive docstring for `SVGBuilder` with its shortcomings documented:

```python
class SVGBuilder(Generatable):
    """A builder class for programmatically creating SVG graphics with styles, patterns, layers, and animations.
    
    SVGBuilder provides a high-level API for constructing complex SVG documents by managing:
    - CSS styles with pseudo-classes (hover, active, etc.)
    - Reusable definitions (patterns, gradients, clip paths)
    - Named layers for organizing content
    - CSS-based animations (fade, slide, scale)
    - Text elements (titles, subtitles, legends)
    
    The builder uses a fluent interface pattern, allowing method chaining for concise code.
    
    Attributes:
        styles (dict): CSS style definitions keyed by class name
        definitions (List[Generatable]): Reusable SVG elements (patterns, gradients, etc.)
        layers (List[SVGLayer]): Named groups of SVG content
        width (int): SVG viewport width in pixels (default: 300)
        height (int): SVG viewport height in pixels (default: 300)
        blockTag (str): Root element tag name (default: "svg", can be "pattern")
        attributes (dict): Additional attributes for the root element
        title (str): Document title (default: "Untitled")
        comment (str): Optional XML comment in header
        animations (dict): Layer-name to LayerAnimation mappings
        
    Examples:
        Basic usage:
        >>> builder = SVGBuilder()
        >>> builder.width = 400
        >>> builder.height = 300
        >>> builder.adjust("background", '<rect x="0" y="0" width="400" height="300" fill="blue"/>')
        >>> svg_xml = builder.xml()
        
        With styles and patterns:
        >>> loader = SVGPatternLoader(patterns_dir="patterns")
        >>> pattern, path = loader.stylizedPattern('ball.svg', patName="water")
        >>> builder.add_definition(pattern)
        >>> water_style = StyleCSS("water", fill=path, stroke="#0D47A1")
        >>> builder.add_style(water_style)
        >>> builder.adjust("lake", '<polygon points="..." class="water"/>')
        
        With animations:
        >>> builder.animate_layer("background", animation_type="fade", duration=1.0)
        >>> builder.animate_cascade(duration=0.8, stagger=0.2)
        
        Method chaining:
        >>> (builder
        ...     .add_title("My Map", subtitle="Interactive SVG")
        ...     .animate_cascade(duration=1.0, stagger=0.3)
        ...     .show())
        
    Known Limitations:
        1. **Pattern ID Conflicts**: If multiple patterns share the same ID, only the first 
           is kept. The add_definition() method silently skips duplicates, which can cause
           unexpected behavior if you intended to update a pattern.
           
        2. **Layer Order**: Layers are rendered in the order they're added. There's no 
           z-index control or explicit layer reordering API beyond manual list manipulation.
           
        3. **Animation Limitations**: 
           - Only supports CSS animations (no SMIL <animate> elements)
           - Cannot animate individual elements within a layer
           - No animation callbacks or events
           - Limited to predefined animation types (fade, slide, scale)
           
        4. **Style Deduplication**: add_style() uses dict keys, so adding a style with the
           same name replaces the previous one without warning.
           
        5. **No Validation**: The builder doesn't validate:
           - SVG syntax in layer bodies
           - CSS property names or values
           - Coordinate bounds or viewport overflow
           - Class name references (can reference non-existent classes)
           
        6. **Memory Efficiency**: Large SVGs with many layers/definitions are kept entirely
           in memory. No streaming or incremental generation support.
           
        7. **Limited Text Support**: Text layout is manual - no automatic wrapping, alignment
           helpers beyond text-anchor, or multi-line text utilities.
           
        8. **Browser Compatibility**: Generated animations use modern CSS features that may
           not work in older browsers (IE11, older mobile browsers).
           
        9. **Pattern Scaling**: Pattern transforms are global - can't easily apply different
           scales to the same pattern in different contexts without creating duplicate patterns.
           
        10. **No Undo/Redo**: Changes to layers, styles, or definitions are destructive with
            no built-in history or rollback mechanism.
    
    See Also:
        StyleCSS: CSS style management with pseudo-classes
        SVGLayer: Named layer/group management
        SVGPatternLoader: Loading SVG files as reusable patterns
        LayerAnimation: Animation configuration
        
    Notes:
        - Use adjust() to add/update layers by name
        - Use add_definition() for reusable elements (avoids duplication by ID)
        - Use add_style() for CSS classes
        - Call xml() for complete XML document or generate() for just the SVG element
        - Use show() in Jupyter/solveit for interactive preview
    """
    
    def __init__(self):
        self.styles = {}
        self.definitions: List[Generatable] = []
        self.layers: List[SVGLayer] = [SVGLayer("root", "")]
        self.width = 300
        self.height = 300
        self.blockTag = "svg"
        self.attributes = {} 
        self.title = "Untitled"
        self.comment = None
        self.animations = {}
        self.animation_counter = 0
```

This docstring provides:
1. **Clear overview** of what the class does
2. **Attribute documentation** with types and defaults
3. **Multiple examples** showing different use cases
4. **Comprehensive limitations list** with 10 specific shortcomings
5. **See Also** section for related classes
6. **Usage notes** for best practices

Would you like me to add docstrings for specific methods as well?